In [20]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

In [21]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_bin.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

In [25]:
#origin = 'n1'
origin = 'n52'
dests = ['n22', 'n66']
sys_surv_st = 2

def s_fun(comps_st):
    travel_time, sys_st, info = fbg.eval_travel_time_to_nearest(comps_st, G_base, origin, dests,
                                                         avg_speed=60, # km/h
                                                         target_max = [1.5, 0.5], # hours: it shouldn't take longer than this compared to the original travel time
                                                         length_attr = 'length_km')
    if sys_st >= sys_surv_st:
       path = info['path_filtered_edges'] 
       min_comps_st = {eid: ('>=', 1) for eid in path} # edges in the path are working
       min_comps_st['sys'] = ('>=', sys_st) # system edge is also working
    else:
        min_comps_st = None
    return travel_time, sys_st, min_comps_st

row_names = list(edges.keys()) 
n_state = 2 # binary states of components

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)


Check system function

In [26]:
comps_st = {eid: 1 for eid in edges.keys()}
travel_time, sys_st, info = s_fun(comps_st)
print(f"travel_time: {travel_time}, sys_st: {sys_st}, info: {info}")

travel_time: 0.7774131878455196, sys_st: 2, info: {'e0097': ('>=', 1), 'e0081': ('>=', 1), 'e0044': ('>=', 1), 'sys': ('>=', 2)}


In [32]:
import importlib
importlib.reload(tsum)

<module 'tsum.tsum' from 'c:\\Users\\jb622s\\git\\tsum\\tsum\\tsum.py'>

In [33]:
result = tsum.run_rule_extraction_by_mcs(
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    output_dir="tsum_res",
    unk_prob_thres=5e-3,
    sys_surv_st=sys_surv_st
) 

---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 2, System value: 0.7774131878455196. Total samples: 100000.
New rule (No. of conditions: 3): {'e0097': ('>=', 1), 'e0081': ('>=', 1), 'e0044': ('>=', 1), 'sys': ('>=', 2)}
Updated sys_vals: [0.777]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 1, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 2, System value: 1.0525899452669656. Total samples: 100000.
New rule (No. of conditions: 4): {'e0097': ('>=', 1), 'e0095': ('>=', 1), 'e0058': ('>=', 1), 'e0043': ('>=', 1), 'sys': ('>=', 2)}
Updated sys_vals: [0.777, 1.053]
---
Round: 3, Unk. prob.: 2.725e-01
Surv probs: 7.275e-01, Fail probs: 0.000e+00
